In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
from sklearn.preprocessing import MinMaxScaler
from matplotlib import font_manager, rc

font_name = font_manager.FontProperties(fname="/usr/share/fonts/truetype/nanum/NanumGothic_Coding_Bold.ttf").get_name()
rc('font', family=font_name)

%matplotlib tk

tf.set_random_seed(777)  # reproducibility

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')

import matplotlib.pyplot as plt

In [2]:
# train Parameters
seq_length = 9
data_dim = 4
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 901

scaler = MinMaxScaler(feature_range=(0,1))
scaler2 = MinMaxScaler(feature_range=(0, 1))

# Open, High, Low, Volume, Close
xy = np.loadtxt('./v2data/uvb/train_v2_data.csv', delimiter=',')
x = scaler.fit_transform(xy[:, 0:-1])
y = scaler2.fit_transform(xy[:, [-1]])  # Close as label

print(xy[:, 0:-1])
print(xy[:, [-1]])

# build a dataset
dataX = []
dataY = []

for i in range(0, len(x) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    dataX.append(_x)
    dataY.append(_y)

xy2 = np.loadtxt('./sun_data/2018-02-08.csv',delimiter=',')
print(xy2)

x2 = scaler.fit_transform(xy2) # 데이터 없을때
# x2 = scaler.fit_transform(xy2[:, 0:-1]) # 데이터 있을때
# y2 = scaler2.fit_transform(xy2[:, [-1]]) #데이터 있을때

dataX2 = []
# dataY2 = [] #데이터 있을때

for i in range(0, len(x2) - seq_length):
    _x2 = x2[i:i+seq_length]
#     _y2 = y2[i+seq_length] #데이터 있을때
    dataX2.append(_x2)
#     dataY2.append(_y2) #데이터 있을때
# train/test split

train_size = len(dataX)
test_size = len(dataX2)

print(len(dataX))
print(len(dataX2))

trainX, testX = np.array(dataX[0:train_size]),np.array(dataX2[0:test_size])
trainY = np.array(dataY[0:train_size])

[[ 61.  -4.  53.  19.]
 [ 61.  -4.  53.  19.]
 [ 61.  -4.  53.  19.]
 ...
 [271. -21. 330. -11.]
 [271. -21. 330. -11.]
 [271. -21. 330. -11.]]
[[-2.87e-05]
 [-2.03e-05]
 [-1.43e-05]
 ...
 [ 1.48e-06]
 [-2.14e-06]
 [ 3.16e-06]]
[[ 86. -29. 321. -15.]
 [ 86. -29. 321. -15.]
 [ 87. -29. 321. -15.]
 ...
 [268. -23. 322. -14.]
 [268. -23. 322. -14.]
 [268. -23. 322. -14.]]
8100
892


In [3]:
# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))
    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    
    test_predict = scaler2.inverse_transform(test_predict)
#     dataY2 = scaler2.inverse_transform(dataY2) # 데이터 있을때

[step: 0] loss: 1930.05078125
[step: 1] loss: 1303.6395263671875
[step: 2] loss: 950.427734375
[step: 3] loss: 810.900634765625
[step: 4] loss: 778.6639404296875
[step: 5] loss: 751.0137329101562
[step: 6] loss: 687.8187255859375
[step: 7] loss: 599.82275390625
[step: 8] loss: 513.1419677734375
[step: 9] loss: 448.2466125488281
[step: 10] loss: 412.2690124511719
[step: 11] loss: 397.8635559082031
[step: 12] loss: 387.068359375
[step: 13] loss: 362.582763671875
[step: 14] loss: 320.3615417480469
[step: 15] loss: 273.91351318359375
[step: 16] loss: 248.537353515625
[step: 17] loss: 259.078125
[step: 18] loss: 278.83966064453125
[step: 19] loss: 267.6593322753906
[step: 20] loss: 228.976806640625
[step: 21] loss: 194.00473022460938
[step: 22] loss: 179.64405822753906
[step: 23] loss: 176.44979858398438
[step: 24] loss: 166.70677185058594
[step: 25] loss: 144.7452392578125
[step: 26] loss: 120.10762786865234
[step: 27] loss: 107.55732727050781
[step: 28] loss: 111.1275863647461
[step: 29] 

[step: 229] loss: 13.314630508422852
[step: 230] loss: 13.30880069732666
[step: 231] loss: 13.303089141845703
[step: 232] loss: 13.297504425048828
[step: 233] loss: 13.292037010192871
[step: 234] loss: 13.286691665649414
[step: 235] loss: 13.281458854675293
[step: 236] loss: 13.276333808898926
[step: 237] loss: 13.271307945251465
[step: 238] loss: 13.266388893127441
[step: 239] loss: 13.26156234741211
[step: 240] loss: 13.256832122802734
[step: 241] loss: 13.252193450927734
[step: 242] loss: 13.247641563415527
[step: 243] loss: 13.243170738220215
[step: 244] loss: 13.238781929016113
[step: 245] loss: 13.23446273803711
[step: 246] loss: 13.23022747039795
[step: 247] loss: 13.226064682006836
[step: 248] loss: 13.221967697143555
[step: 249] loss: 13.217934608459473
[step: 250] loss: 13.213967323303223
[step: 251] loss: 13.210064888000488
[step: 252] loss: 13.206221580505371
[step: 253] loss: 13.202425956726074
[step: 254] loss: 13.198692321777344
[step: 255] loss: 13.195013999938965
[step

[step: 453] loss: 12.712242126464844
[step: 454] loss: 12.710062980651855
[step: 455] loss: 12.707876205444336
[step: 456] loss: 12.705694198608398
[step: 457] loss: 12.703511238098145
[step: 458] loss: 12.701325416564941
[step: 459] loss: 12.699139595031738
[step: 460] loss: 12.696958541870117
[step: 461] loss: 12.694770812988281
[step: 462] loss: 12.69258975982666
[step: 463] loss: 12.690402030944824
[step: 464] loss: 12.688216209411621
[step: 465] loss: 12.686037063598633
[step: 466] loss: 12.683846473693848
[step: 467] loss: 12.681666374206543
[step: 468] loss: 12.679475784301758
[step: 469] loss: 12.677291870117188
[step: 470] loss: 12.675097465515137
[step: 471] loss: 12.67291259765625
[step: 472] loss: 12.670727729797363
[step: 473] loss: 12.66853141784668
[step: 474] loss: 12.666348457336426
[step: 475] loss: 12.66416072845459
[step: 476] loss: 12.661969184875488
[step: 477] loss: 12.65977954864502
[step: 478] loss: 12.657588958740234
[step: 479] loss: 12.655396461486816
[step:

[step: 676] loss: 12.203866958618164
[step: 677] loss: 12.201435089111328
[step: 678] loss: 12.198994636535645
[step: 679] loss: 12.196558952331543
[step: 680] loss: 12.19411563873291
[step: 681] loss: 12.19167423248291
[step: 682] loss: 12.189229011535645
[step: 683] loss: 12.186779022216797
[step: 684] loss: 12.184328079223633
[step: 685] loss: 12.181877136230469
[step: 686] loss: 12.179421424865723
[step: 687] loss: 12.17696475982666
[step: 688] loss: 12.174510955810547
[step: 689] loss: 12.172046661376953
[step: 690] loss: 12.169587135314941
[step: 691] loss: 12.167121887207031
[step: 692] loss: 12.164651870727539
[step: 693] loss: 12.162179946899414
[step: 694] loss: 12.159708023071289
[step: 695] loss: 12.157235145568848
[step: 696] loss: 12.154760360717773
[step: 697] loss: 12.152280807495117
[step: 698] loss: 12.149797439575195
[step: 699] loss: 12.14731502532959
[step: 700] loss: 12.14482593536377
[step: 701] loss: 12.142339706420898
[step: 702] loss: 12.13984203338623
[step: 

[step: 900] loss: 11.617170333862305


In [6]:
# Plot predictions
plt.title('2018-02-08 predict')
# plt.plot(dataY2, label='row data') # 데이터 있을때
plt.plot(test_predict, label='predict')
plt.xlabel("Time Period")
plt.ylabel("uvb")
# plt.ylim(0, 140000)
plt.xticks(np.arange(0, 901, step = 56), ['5', '6', '7', '8', '9', '10', '11', '12', '1', '2', '3', '4', '5', '6', '7', '8']) # v2 모델
plt.yticks(np.arange(-0.25,2.25, step=0.25))
plt.grid(True)
plt.legend(loc='upper right')
plt.show()